In [1]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='
p5100 ={'serviceKey' :serviceKey,
             'routeId':'200000115','stationId':'228000703'}#5100 외대
p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]
    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./{FileName}')        # 동일 폴더에 파일이 저장될 거임
    return df

In [2]:

past=4
pBus=Bus(p7000)
for x in range(720): # 12시간
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
        
    except requests.Timeout as err:
        continue
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'8.13.csv')
mod_df

0 번째 시도:  [['2022-08-13 12:05:24.835', [4, 11, 24, 27, 37, 44, 50, 60, 69, 79], 0]]
1 번째 시도:  [['2022-08-13 12:05:24.835', [4, 11, 24, 27, 37, 44, 50, 60, 69, 79], 0], ['2022-08-13 12:06:25.981', [5, 12, 25, 27, 38, 44, 51, 60, 70, 79], 0]]
2 번째 시도:  [['2022-08-13 12:05:24.835', [4, 11, 24, 27, 37, 44, 50, 60, 69, 79], 0], ['2022-08-13 12:06:25.981', [5, 12, 25, 27, 38, 44, 51, 60, 70, 79], 0], ['2022-08-13 12:07:26.143', [6, 13, 25, 28, 39, 45, 51, 61, 71, 79], 0]]
3 번째 시도:  [['2022-08-13 12:05:24.835', [4, 11, 24, 27, 37, 44, 50, 60, 69, 79], 0], ['2022-08-13 12:06:25.981', [5, 12, 25, 27, 38, 44, 51, 60, 70, 79], 0], ['2022-08-13 12:07:26.143', [6, 13, 25, 28, 39, 45, 51, 61, 71, 79], 0], ['2022-08-13 12:08:26.289', [7, 13, 26, 28, 39, 45, 51, 62, 71, 79], 0]]
4 번째 시도:  [['2022-08-13 12:05:24.835', [4, 11, 24, 27, 37, 44, 50, 60, 69, 79], 0], ['2022-08-13 12:06:25.981', [5, 12, 25, 27, 38, 44, 51, 60, 70, 79], 0], ['2022-08-13 12:07:26.143', [6, 13, 25, 28, 39, 45, 51, 61, 71, 79], 

15 번째 시도:  [['2022-08-13 12:05:24.835', [4, 11, 24, 27, 37, 44, 50, 60, 69, 79], 0], ['2022-08-13 12:06:25.981', [5, 12, 25, 27, 38, 44, 51, 60, 70, 79], 0], ['2022-08-13 12:07:26.143', [6, 13, 25, 28, 39, 45, 51, 61, 71, 79], 0], ['2022-08-13 12:08:26.289', [7, 13, 26, 28, 39, 45, 51, 62, 71, 79], 0], ['2022-08-13 12:09:26.467', [8, 13, 26, 29, 40, 45, 51, 63, 72, 79], 0], ['2022-08-13 12:10:26.598', [8, 14, 26, 29, 40, 46, 52, 63, 73, 79], 0], ['2022-08-13 12:11:26.729', [9, 14, 26, 30, 40, 46, 52, 63, 74], 0], ['2022-08-13 12:12:27.969', [9, 15, 26, 31, 40, 47, 52, 64, 76], 0], ['2022-08-13 12:13:28.116', [10, 16, 27, 31, 41, 48, 53, 65, 76], 0], ['2022-08-13 12:14:28.278', [10, 16, 27, 31, 41, 48, 53, 65, 76], 0], ['2022-08-13 12:15:28.502', [1, 10, 17, 27, 32, 42, 48, 54, 65, 77], 1], ['2022-08-13 12:16:28.648', [3, 10, 17, 28, 32, 42, 49, 54, 66, 78], 0], ['2022-08-13 12:17:28.810', [3, 11, 18, 28, 33, 43, 50, 55, 66, 79], 0], ['2022-08-13 12:18:28.972', [3, 11, 18, 29, 33, 44, 5

22 번째 시도:  [['2022-08-13 12:05:24.835', [4, 11, 24, 27, 37, 44, 50, 60, 69, 79], 0], ['2022-08-13 12:06:25.981', [5, 12, 25, 27, 38, 44, 51, 60, 70, 79], 0], ['2022-08-13 12:07:26.143', [6, 13, 25, 28, 39, 45, 51, 61, 71, 79], 0], ['2022-08-13 12:08:26.289', [7, 13, 26, 28, 39, 45, 51, 62, 71, 79], 0], ['2022-08-13 12:09:26.467', [8, 13, 26, 29, 40, 45, 51, 63, 72, 79], 0], ['2022-08-13 12:10:26.598', [8, 14, 26, 29, 40, 46, 52, 63, 73, 79], 0], ['2022-08-13 12:11:26.729', [9, 14, 26, 30, 40, 46, 52, 63, 74], 0], ['2022-08-13 12:12:27.969', [9, 15, 26, 31, 40, 47, 52, 64, 76], 0], ['2022-08-13 12:13:28.116', [10, 16, 27, 31, 41, 48, 53, 65, 76], 0], ['2022-08-13 12:14:28.278', [10, 16, 27, 31, 41, 48, 53, 65, 76], 0], ['2022-08-13 12:15:28.502', [1, 10, 17, 27, 32, 42, 48, 54, 65, 77], 1], ['2022-08-13 12:16:28.648', [3, 10, 17, 28, 32, 42, 49, 54, 66, 78], 0], ['2022-08-13 12:17:28.810', [3, 11, 18, 28, 33, 43, 50, 55, 66, 79], 0], ['2022-08-13 12:18:28.972', [3, 11, 18, 29, 33, 44, 5

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /6410000/busarrivalservice/getBusArrivalItem?serviceKey=bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ%3D%3D&routeId=200000112&stationId=228000703 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000012DE66A1940>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [3]:

past=4
pBus=Bus(p7000)
for x in range(720): # 12시간
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
        
    except requests.Timeout as err:
        continue
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'8.13.csv')
mod_df

0 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0]]
1 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0]]
2 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0]]
3 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0]]
4 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0], ['2022-08-13 13:26:25

17 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0], ['2022-08-13 13:26:25.908', [20, 27, 34, 43, 48, 63, 71, 78], 0], ['2022-08-13 13:27:33.819', [21, 28, 34, 44, 48, 63, 72, 79], 0], ['2022-08-13 13:28:33.981', [22, 28, 35, 44, 49, 63, 73], 0], ['2022-08-13 13:29:49.406', [23, 28, 36, 45, 50, 64, 75], 0], ['2022-08-13 13:30:54.349', [1, 23, 29, 37, 45, 50, 65, 76], 1], ['2022-08-13 13:32:09.087', [3, 24, 29, 38, 45, 51, 65, 76], 0], ['2022-08-13 13:33:09.311', [3, 25, 29, 38, 46, 51, 65, 77], 0], ['2022-08-13 13:34:09.630', [3, 25, 30, 39, 46, 51, 67, 78], 0], ['2022-08-13 13:35:09.776', [4, 26, 30, 39, 47, 51, 67, 79], 0], ['2022-08-13 13:36:11.641', [5, 26, 31, 39, 47, 51, 68], 0], ['2022-08-13 13:37:15.928', [5, 26, 31, 40, 48, 52, 68], 0], ['2022-08-13 13:38:16

24 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0], ['2022-08-13 13:26:25.908', [20, 27, 34, 43, 48, 63, 71, 78], 0], ['2022-08-13 13:27:33.819', [21, 28, 34, 44, 48, 63, 72, 79], 0], ['2022-08-13 13:28:33.981', [22, 28, 35, 44, 49, 63, 73], 0], ['2022-08-13 13:29:49.406', [23, 28, 36, 45, 50, 64, 75], 0], ['2022-08-13 13:30:54.349', [1, 23, 29, 37, 45, 50, 65, 76], 1], ['2022-08-13 13:32:09.087', [3, 24, 29, 38, 45, 51, 65, 76], 0], ['2022-08-13 13:33:09.311', [3, 25, 29, 38, 46, 51, 65, 77], 0], ['2022-08-13 13:34:09.630', [3, 25, 30, 39, 46, 51, 67, 78], 0], ['2022-08-13 13:35:09.776', [4, 26, 30, 39, 47, 51, 67, 79], 0], ['2022-08-13 13:36:11.641', [5, 26, 31, 39, 47, 51, 68], 0], ['2022-08-13 13:37:15.928', [5, 26, 31, 40, 48, 52, 68], 0], ['2022-08-13 13:38:16

29 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0], ['2022-08-13 13:26:25.908', [20, 27, 34, 43, 48, 63, 71, 78], 0], ['2022-08-13 13:27:33.819', [21, 28, 34, 44, 48, 63, 72, 79], 0], ['2022-08-13 13:28:33.981', [22, 28, 35, 44, 49, 63, 73], 0], ['2022-08-13 13:29:49.406', [23, 28, 36, 45, 50, 64, 75], 0], ['2022-08-13 13:30:54.349', [1, 23, 29, 37, 45, 50, 65, 76], 1], ['2022-08-13 13:32:09.087', [3, 24, 29, 38, 45, 51, 65, 76], 0], ['2022-08-13 13:33:09.311', [3, 25, 29, 38, 46, 51, 65, 77], 0], ['2022-08-13 13:34:09.630', [3, 25, 30, 39, 46, 51, 67, 78], 0], ['2022-08-13 13:35:09.776', [4, 26, 30, 39, 47, 51, 67, 79], 0], ['2022-08-13 13:36:11.641', [5, 26, 31, 39, 47, 51, 68], 0], ['2022-08-13 13:37:15.928', [5, 26, 31, 40, 48, 52, 68], 0], ['2022-08-13 13:38:16

34 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0], ['2022-08-13 13:26:25.908', [20, 27, 34, 43, 48, 63, 71, 78], 0], ['2022-08-13 13:27:33.819', [21, 28, 34, 44, 48, 63, 72, 79], 0], ['2022-08-13 13:28:33.981', [22, 28, 35, 44, 49, 63, 73], 0], ['2022-08-13 13:29:49.406', [23, 28, 36, 45, 50, 64, 75], 0], ['2022-08-13 13:30:54.349', [1, 23, 29, 37, 45, 50, 65, 76], 1], ['2022-08-13 13:32:09.087', [3, 24, 29, 38, 45, 51, 65, 76], 0], ['2022-08-13 13:33:09.311', [3, 25, 29, 38, 46, 51, 65, 77], 0], ['2022-08-13 13:34:09.630', [3, 25, 30, 39, 46, 51, 67, 78], 0], ['2022-08-13 13:35:09.776', [4, 26, 30, 39, 47, 51, 67, 79], 0], ['2022-08-13 13:36:11.641', [5, 26, 31, 39, 47, 51, 68], 0], ['2022-08-13 13:37:15.928', [5, 26, 31, 40, 48, 52, 68], 0], ['2022-08-13 13:38:16

39 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0], ['2022-08-13 13:26:25.908', [20, 27, 34, 43, 48, 63, 71, 78], 0], ['2022-08-13 13:27:33.819', [21, 28, 34, 44, 48, 63, 72, 79], 0], ['2022-08-13 13:28:33.981', [22, 28, 35, 44, 49, 63, 73], 0], ['2022-08-13 13:29:49.406', [23, 28, 36, 45, 50, 64, 75], 0], ['2022-08-13 13:30:54.349', [1, 23, 29, 37, 45, 50, 65, 76], 1], ['2022-08-13 13:32:09.087', [3, 24, 29, 38, 45, 51, 65, 76], 0], ['2022-08-13 13:33:09.311', [3, 25, 29, 38, 46, 51, 65, 77], 0], ['2022-08-13 13:34:09.630', [3, 25, 30, 39, 46, 51, 67, 78], 0], ['2022-08-13 13:35:09.776', [4, 26, 30, 39, 47, 51, 67, 79], 0], ['2022-08-13 13:36:11.641', [5, 26, 31, 39, 47, 51, 68], 0], ['2022-08-13 13:37:15.928', [5, 26, 31, 40, 48, 52, 68], 0], ['2022-08-13 13:38:16

43 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0], ['2022-08-13 13:26:25.908', [20, 27, 34, 43, 48, 63, 71, 78], 0], ['2022-08-13 13:27:33.819', [21, 28, 34, 44, 48, 63, 72, 79], 0], ['2022-08-13 13:28:33.981', [22, 28, 35, 44, 49, 63, 73], 0], ['2022-08-13 13:29:49.406', [23, 28, 36, 45, 50, 64, 75], 0], ['2022-08-13 13:30:54.349', [1, 23, 29, 37, 45, 50, 65, 76], 1], ['2022-08-13 13:32:09.087', [3, 24, 29, 38, 45, 51, 65, 76], 0], ['2022-08-13 13:33:09.311', [3, 25, 29, 38, 46, 51, 65, 77], 0], ['2022-08-13 13:34:09.630', [3, 25, 30, 39, 46, 51, 67, 78], 0], ['2022-08-13 13:35:09.776', [4, 26, 30, 39, 47, 51, 67, 79], 0], ['2022-08-13 13:36:11.641', [5, 26, 31, 39, 47, 51, 68], 0], ['2022-08-13 13:37:15.928', [5, 26, 31, 40, 48, 52, 68], 0], ['2022-08-13 13:38:16

46 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0], ['2022-08-13 13:26:25.908', [20, 27, 34, 43, 48, 63, 71, 78], 0], ['2022-08-13 13:27:33.819', [21, 28, 34, 44, 48, 63, 72, 79], 0], ['2022-08-13 13:28:33.981', [22, 28, 35, 44, 49, 63, 73], 0], ['2022-08-13 13:29:49.406', [23, 28, 36, 45, 50, 64, 75], 0], ['2022-08-13 13:30:54.349', [1, 23, 29, 37, 45, 50, 65, 76], 1], ['2022-08-13 13:32:09.087', [3, 24, 29, 38, 45, 51, 65, 76], 0], ['2022-08-13 13:33:09.311', [3, 25, 29, 38, 46, 51, 65, 77], 0], ['2022-08-13 13:34:09.630', [3, 25, 30, 39, 46, 51, 67, 78], 0], ['2022-08-13 13:35:09.776', [4, 26, 30, 39, 47, 51, 67, 79], 0], ['2022-08-13 13:36:11.641', [5, 26, 31, 39, 47, 51, 68], 0], ['2022-08-13 13:37:15.928', [5, 26, 31, 40, 48, 52, 68], 0], ['2022-08-13 13:38:16

49 번째 시도:  [['2022-08-13 13:22:25.025', [18, 26, 32, 41, 46, 59, 69, 75], 0], ['2022-08-13 13:23:25.312', [19, 27, 32, 41, 46, 60, 69, 76], 0], ['2022-08-13 13:24:25.459', [19, 27, 33, 42, 47, 60, 70, 77], 0], ['2022-08-13 13:25:25.730', [20, 27, 33, 42, 48, 61, 71, 77], 0], ['2022-08-13 13:26:25.908', [20, 27, 34, 43, 48, 63, 71, 78], 0], ['2022-08-13 13:27:33.819', [21, 28, 34, 44, 48, 63, 72, 79], 0], ['2022-08-13 13:28:33.981', [22, 28, 35, 44, 49, 63, 73], 0], ['2022-08-13 13:29:49.406', [23, 28, 36, 45, 50, 64, 75], 0], ['2022-08-13 13:30:54.349', [1, 23, 29, 37, 45, 50, 65, 76], 1], ['2022-08-13 13:32:09.087', [3, 24, 29, 38, 45, 51, 65, 76], 0], ['2022-08-13 13:33:09.311', [3, 25, 29, 38, 46, 51, 65, 77], 0], ['2022-08-13 13:34:09.630', [3, 25, 30, 39, 46, 51, 67, 78], 0], ['2022-08-13 13:35:09.776', [4, 26, 30, 39, 47, 51, 67, 79], 0], ['2022-08-13 13:36:11.641', [5, 26, 31, 39, 47, 51, 68], 0], ['2022-08-13 13:37:15.928', [5, 26, 31, 40, 48, 52, 68], 0], ['2022-08-13 13:38:16

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /6410000/busarrivalservice/getBusArrivalItem?serviceKey=bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ%3D%3D&routeId=200000112&stationId=228000703 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000012DE830A280>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))